# 한국 거래소에서 시가총액 데이터 수집
* KRX에서 상장 기업의 시가총액 데이터를 기간별로 수집하는 방법에 대해서 알아보겠습니다. 
* selenium, BeautifulSoup, requests, json 라이브러리를 사용하여 진행하겠습니다. 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd 
import requests
import json
import time

* Chrome를 실제로 운전하게 될 driver라는 웹 드라이버 객체를 생성합니다. 
    > Chrome, Firefox, PhantomJS등 각각의 브라우저마다 드라이버가 다르고, 호출 함수도 다릅니다. 
* 드라이버에게 파이썬 코드로 우리가 웹에서 하는 행동을 지시할 수 있습니다. 
* chromedriver.exe 파일이 저장되어 있는 경로를 Chrome() 메소드에 전달 

In [ ]:
driver = webdriver.Chrome("C:\chromedriver\chromedriver.exe")

* get() 메소드를 사용하여 사용자가 원하는 주소로 이동합니다.
* 이동 뒤에 sleep() 함수를 사용하여 페이지가 완전히 로딩될 때까지 기다립니다. 

In [ ]:
# 종합 시황 다운로드 받기 
driver.get('http://marketdata.krx.co.kr/mdi#document=0101')
# 5초간 기다림
time.sleep(5)

* find_element_by_css_selector 메소드는 selector에서 복사한 값을 기준으로 해당 지정한 곳의 정보를 찾습니다.
* 찾고 난 뒤에 텍스트 상자라면 clear()를 사용하여 값을 삭제할수도, 버튼이라면 click()를 사용하여 실제 클릭을 진행할 수 있습니다.
* 아래 예제는 문서 3개를 다운로드를 자동으로 받는 예제입니다. click() 함수를 사용하여 자동으로 다운로드 합니다.

In [ ]:
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(1) > td.attach-td").click()
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(1) > td.attach-td > div > div > ul > li > a").click()

In [ ]:
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(2) > td.attach-td > div > button").click()           
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(2) > td.attach-td > div > div > ul > li > a").click()      

In [ ]:
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(3) > td.attach-td > div > button").click()           
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(3) > td.attach-td > div > div > ul > li > a").click()       

* 조회 기간의 글자를 clear() 메소드를 사용하여 깨끗하기 지웁니다. 
* 그리고 send_keys() 메소드를 사용하여 내가 원하는 날짜 값을 전달합니다. 

In [ ]:
driver.find_element_by_css_selector('#fromdate1679091c5a880faf6fb5e6087eb1b2dc').clear()

In [ ]:
driver.find_element_by_css_selector('#fromdate1679091c5a880faf6fb5e6087eb1b2dc').send_keys('20190628')

* 자동화를 진행하게 된다면 20190628처럼 사람이 직접 입력하지 않고 자동으로 만들어야 합니다. 
* 우리가 이전 시간에 배웠던 datetime 라이브러리의 date 클래스의 today() 메소드를 사용하여 현재 날짜를 자동으로 가져옵니다.

In [ ]:
import datetime

In [ ]:
datetime.date.today()

* str() 함수를 사용하면 문자열로 변경할 수 있고, 문자열 메소드인 replace()를 사용하면 -를 제거할 수 있습니다.

In [ ]:
str(datetime.date.today()).replace("-", "")

* date객체에서 strftime() 메소드를 사용하면 문자열 변환을 포맷에 맞게 출력할 수 있습니다.

In [ ]:
today = datetime.date.today()

* %Y : year
* %m : Month
* %d : day

In [ ]:
today.strftime("%Y%m%d")

# 시가총액 데이터 다운로드 받기 

* 시가 총액을 다운로드 받을 수 있는 사이트로 이동합니다.
* http://marketdata.krx.co.kr/mdi#document=040402

In [19]:
head = {'Host': 'marketdata.krx.co.kr',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'
        }

In [47]:
otp_url = "http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx"

data = {'bld': 'MKD/04/0404/04040200/mkd04040200_01',
        'name': 'form'}

In [48]:
otp_r = requests.get(otp_url, params=data, headers=head)

* requests.get() 메소드를 통해서 아래 주소를 접근

In [49]:
otp_r.url

'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?bld=MKD%2F04%2F0404%2F04040200%2Fmkd04040200_01&name=form'

* 응답받은 값을 확인

In [50]:
otp_r.text

'MfqYzeUrwxtQn7deVWIdv5KFsdEOsCBJ4X+2QZ/7+/ZqqXv8unnqt/mtyrYtOqsTZrWE3MY3SKJ6BXxz2XqrMDT6DBSsSfOZBUQ/z6dJwv0DftdNLOw7V7SVn7Pt5lKMSYcJZaP38mVibamRrDEQnPS5ycdabP/RYxT+yqfvspk='

* post 방식

In [51]:
market_cap = 'http://marketdata.krx.co.kr/contents/MKD/99/MKD99000001.jspx'

In [52]:
payload = {     'market_gubun': 'ALL',
                'sect_tp_cd': 'ALL',
                'schdate': '20200703',
                'pagePath': '/contents/MKD/04/0404/04040200/MKD04040200.jsp',
                'curPage': '1'
               #'bldcode' : 'MKD/04/0404/04040200/mkd04040200_02'
           }
payload['code'] = otp_r.text

In [53]:
post_code = requests.post(market_cap,  data=payload, headers=head)

In [54]:
post_code.text

'{"시가총액 상하위":[{"rn":"1","isu_cd":"005930","kor_shrt_isu_nm":"삼성전자","isu_cur_pr":"53,600","fluc_tp_cd":"1","prv_dd_cmpr":"700","updn_rate":"1.3","isu_tr_vl":"11,833,037","isu_tr_amt":"630,397,097,900","opnprc":"53,000","hgprc":"53,600","lwprc":"52,700","cur_pr_tot_amt":"319,980,344,680,000","tot_amt_per":"18.38","lst_stk_vl":"5,969,782,550","f1":"","f2":"","totCnt":"2472"},{"rn":"2","isu_cd":"000660","kor_shrt_isu_nm":"SK하이닉스","isu_cur_pr":"85,400","fluc_tp_cd":"1","prv_dd_cmpr":"1,000","updn_rate":"1.2","isu_tr_vl":"2,188,927","isu_tr_amt":"185,910,970,600","opnprc":"85,000","hgprc":"85,600","lwprc":"84,500","cur_pr_tot_amt":"62,171,401,971,000","tot_amt_per":"3.57","lst_stk_vl":"728,002,365","f1":"","f2":"","totCnt":""},{"rn":"3","isu_cd":"207940","kor_shrt_isu_nm":"삼성바이오로직스","isu_cur_pr":"771,000","fluc_tp_cd":"2","prv_dd_cmpr":"15,000","updn_rate":"-1.9","isu_tr_vl":"223,020","isu_tr_amt":"171,902,647,000","opnprc":"795,000","hgprc":"796,000","lwprc":"764,000","cur_pr_tot_amt":"51,0

* for문을 300번까지 돌린 이유는 데이터가 없다면 크기를 확인하고(100) 이하면 종료 로직이 들어 있기 때문에 크게 잡았습니다.
* 종목은 수시로 변경될 수 있기 때문에 숫자를 딱 정하는 것보다 약간 더 크게 만들어 로직을 넣는 것을 추천합니다.

In [55]:
rt_list = []

for cnt in range(1,300):
    otp_r = requests.get(otp_url, params=data, headers=head)
    payload['code'] = otp_r.text
    if(cnt % 10 == 0):
        print (cnt)
    payload['curPage'] = cnt
    rt3 = requests.post(market_cap,  data=payload, headers=head)
    # 응답 받은 데이터가 15보다 크면 입력 아니면 종료 
    if(len(rt3.text) > 15):
        rt_list.append(rt3.text)
    else:
        break

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240


* rt_list의 원소 하나하나는 문자열로 이루어져 있습니다. 눈으로 확인하면 dict 형태이지만(서버에선 json형식으로 전달) 문자열이 때문에 json 라이브러리의 힘을 빌려야 합니다.

* json의 loads() 함수를 사용하여 문자열을 dict 형태로 변환합니다. 그리고 '시가총액 상하위' 키를 가지고 있는 값을 추출하여 DataFrame을 생성합니다.


In [57]:
pd.DataFrame(json.loads(rt_list[0])['시가총액 상하위'])

,rn,isu_cd,kor_shrt_isu_nm,isu_cur_pr,fluc_tp_cd,prv_dd_cmpr,updn_rate,isu_tr_vl,isu_tr_amt,opnprc,hgprc,lwprc,cur_pr_tot_amt,tot_amt_per,lst_stk_vl,f1,f2,totCnt
0,1,005930,삼성전자,"53,600",1,700,1.3,"11,833,037","630,397,097,900","53,000","53,600","52,700","319,980,344,680,000",18.38,"5,969,782,550",,,2472
1,2,000660,SK하이닉스,"85,400",1,"1,000",1.2,"2,188,927","185,910,970,600","85,000","85,600","84,500","62,171,401,971,000",3.57,"728,002,365",,,
2,3,207940,삼성바이오로직스,"771,000",2,"15,000",-1.9,"223,020","171,902,647,000","795,000","796,000","764,000","51,013,215,000,000",2.93,"66,165,000",,,
3,4,035420,NAVER,"277,500",1,500,0.2,"967,298","268,712,672,250","276,500","280,500","274,500","45,583,092,112,500",2.62,"164,263,395",,,
4,5,068270,셀트리온,"309,500",1,"1,000",0.3,"754,165","231,070,318,000","311,500","312,000","300,500","41,763,739,657,500",2.40,"134,939,385",,,
5,6,005935,삼성전자우,"46,900",1,250,0.5,"1,452,190","67,952,160,115","46,700","46,950","46,500","38,593,386,230,000",2.22,"822,886,700",,,
6,7,051910,LG화학,"507,000",1,"11,000",2.2,"440,508","223,608,031,000","509,000","515,000","502,000","35,790,317,901,000",2.06,"70,592,343",,,
7,8,006400,삼성SDI,"381,000",1,"9,500",2.6,"538,722","205,254,685,000","379,500","385,000","378,000","26,199,285,930,000",1.50,"68,764,530",,,
8,9,035720,카카오,"294,000",1,"8,500",3.0,"1,597,485","467,180,461,500","289,500","297,000","284,000","25,813,202,352,000",1.48,"87,800,008",,,
9,10,051900,LG생활건강,"1,380,000",1,"1,000",0.1,"41,664","57,415,539,768","1,377,000","1,389,000","1,357,000","21,553,111,860,000",1.24,"15,618,197",,,


* 이 작업을 for문을 돌려서 concat() 함수(DataFrame을 row단위로 합쳐주는 역할) 호출하여 DataFrame을 total_df라는 하나의 데이터 합칩니다.

In [58]:
total_df = pd.DataFrame()
for x in rt_list:
    rt = json.loads(x)
    total_df = pd.concat([total_df, pd.DataFrame(rt['시가총액 상하위'])])

In [61]:
column = { 'rn'     : '순위', 
           'isu_cd' : '종목코드', 
           'kor_shrt_isu_nm' :'종목명',
           'opnprc' : '시가',
            'hgprc' : '고가',
            'lwprc' : '저가',
            'isu_cur_pr' :'종가',
            'prv_dd_cmpr' : '대비',
            'fluc_tp_cd' : '상승하락',          
            'updn_rate' : '등락률(%)',
            'isu_tr_vl': '거래량(주)',
            'isu_tr_amt' : '거래대금',
            'cur_pr_tot_amt' : '시가총액',
            'tot_amt_per' : '시가총액비중(%)',
            'lst_stk_vl' : '상장주식수',
            'f1' : '외국인보유수량',
            'f2' : '외국인 지분율(%)'}

In [63]:
total_df.rename(columns=column, inplace=True)

* 해당 데이터는 날짜 데이터가 없기 때문에 post()안에 요청했던 날짜로 date라는 컬럼을 생성하고 날짜를 넣어줍니다. 
* 이 작업도 자동화로 해야겠죠? 위의 코드를 바탕으로 만들어보는 것을 권장합니다.

In [65]:
total_df['date']= '2020-07-03'

* 인덱스 값을 초기화합니다.

In [68]:
total_df.reset_index(drop=True, inplace=True)